# Chapter 8 - Data Wrangling: Join, Combine & Reshape

In [1]:
import pandas as pd
import numpy as np

In [5]:
data = pd.Series(np.random.uniform(size=9),
                 [["a", "a", "a", "b", "b", "c", "c", "d", "d"],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.503155
   2    0.241305
   3    0.208224
b  1    0.739326
   3    0.365001
c  1    0.223615
   2    0.111321
d  2    0.255020
   3    0.818771
dtype: float64

In [6]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
data["b"]

1    0.739326
3    0.365001
dtype: float64

In [8]:
data["b":"c"]

b  1    0.739326
   3    0.365001
c  1    0.223615
   2    0.111321
dtype: float64

In [9]:
data.loc[["b", "c"]]

b  1    0.739326
   3    0.365001
c  1    0.223615
   2    0.111321
dtype: float64

In [10]:
data.unstack()

,1,2,3
a,0.503155,0.241305,0.208224
b,0.739326,NaN,0.365001
c,0.223615,0.111321,NaN
d,NaN,0.255020,0.818771


In [12]:
data.unstack().stack()

a  1    0.503155
   2    0.241305
   3    0.208224
b  1    0.739326
   3    0.365001
c  1    0.223615
   2    0.111321
d  2    0.255020
   3    0.818771
dtype: float64

In [13]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[["a", "a", "b", "b"],[1 ,2, 1, 2]],
                    columns=[["Ohio", "Ohio", "Colorado"],
                            ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame.index.names = ["key1", "key2"]

In [16]:
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
frame.index.nlevels

2

In [18]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [20]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                         names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering & Sorting Levels

In [21]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [22]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [23]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [24]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistical by Level

In [25]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [27]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [28]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                     "c": ["one", "one", "one", "two", "two",
                          "two", "two"],
                     "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [29]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [30]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [31]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [32]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 Combining & Mergin Datasets

### Database-Style DataFrame Joins

In [59]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                   "data1": pd.Series(range(7), dtype="Int64")})

In [58]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                   "data2": pd.Series(range(3), dtype="Int64")})

In [60]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [61]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [62]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [63]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [54]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                   "data1": pd.Series(range(7), dtype="Int64")})

In [55]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                 "data2": pd.Series(range(3), dtype="Int64")})

In [56]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [57]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,<NA>
1,b,1,<NA>
2,b,6,<NA>
3,a,2,<NA>
4,a,4,<NA>
5,a,5,<NA>
6,c,3,<NA>
7,,<NA>,0
8,,<NA>,1
9,,<NA>,2


In [65]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [66]:
df1 = pd.DataFrame({"key":["b", "b", "a", "c", "a", "b"],
                   "data1": pd.Series(range(6), dtype="Int64")})

In [67]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                   "data2": pd.Series(range(5), dtype="Int64")})

In [68]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [69]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [71]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [72]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [73]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                    "key2": ["one", "two", "one"],
                    "lval": pd.Series([1, 2, 3], dtype='Int64')})

In [75]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                    "key2": ["one", "one", "one", "two"],
                    "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

In [76]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [77]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [78]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

In [19]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                     "value": pd.Series(range(6), dtype="Int64")})

In [20]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [21]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [22]:
right1

,group_val
a,3.5
b,7.0


In [23]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [24]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [33]:
lefth=pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
                   "key2": [2000, 2001, 2002, 2001, 2002],
                   "data": pd.Series(range(5), dtype="Int64")})

In [34]:
righth_index = pd.MultiIndex.from_arrays(
[
    ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
    [2001, 2000, 2000, 2000, 2001, 2002]
])

In [35]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64", index=righth_index),
                      "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64", index=righth_index)})

In [36]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [37]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [38]:
pd.merge(lefth, righth, left_on=["key1", "key2"],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [39]:
pd.merge(lefth, righth, left_on=["key1", "key2"],right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [46]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")

In [47]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=["b", "c", "d", "e"],
                     columns=["Missouri", "Alabama"]).astype("Int64")

In [48]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [49]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [50]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [53]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [55]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [56]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index=["a", "c", "e", "f"],
                      columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [57]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [58]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### Concatenating Along an Axis

In [59]:
arr = np.arange(12).reshape((3, 4))

In [60]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [61]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [62]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")

In [63]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")

In [64]:
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [65]:
s1

a    0
b    1
dtype: Int64

In [66]:
s2

c    2
d    3
e    4
dtype: Int64

In [67]:
s3

f    5
g    6
dtype: Int64

In [68]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [69]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [70]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [72]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [74]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [77]:
result = pd.concat([s1, s2, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: Int64

In [78]:
result.unstack()

,a,b,c,d,e,f,g
one,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
two,<NA>,<NA>,2,3,4,<NA>,<NA>
three,<NA>,<NA>,<NA>,<NA>,<NA>,5,6


In [80]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [81]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"], columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"], columns=["three", "four"])

In [82]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [84]:
df2

,three,four
a,5,6
c,7,8


In [85]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [86]:
pd.concat({"level1":df1, "level2":df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [87]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"], names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [88]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                  columns=["a", "b", "c", "d"])

In [89]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                  columns=["b", "d", "a"])

In [90]:
df1

,a,b,c,d
0,0.176199,0.853621,0.308272,-0.944403
1,0.083977,-0.175795,-0.398395,0.491733
2,-0.058822,0.439624,0.273408,-0.050955


In [91]:
df2

,b,d,a
0,1.402523,0.450483,-0.269247
1,-0.983261,-0.433740,-0.955415


In [92]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.176199,0.853621,0.308272,-0.944403
1,0.083977,-0.175795,-0.398395,0.491733
2,-0.058822,0.439624,0.273408,-0.050955
3,-0.269247,1.402523,NaN,0.450483
4,-0.955415,-0.983261,NaN,-0.433740


### Combining Data with Overlap

In [93]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=["f", "e", "d", "c", "b", "a"])

In [94]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.], index=["a", "b", "c", "d", "e", "f"])

In [95]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [96]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [97]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [98]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                   "b": [np.nan, 2., np.nan, 6.],
                   "c": range(2, 18, 4)})

In [99]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                   "b": [np.nan, 3., 4., 6., 8.]})

In [100]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [101]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [102]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 Reshaping & Pivoting

### Reshaping with Hierarchical Indexing

In [2]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                   index=pd.Index(["Ohio", "Colorado"], name="state"),
                   columns=pd.Index(["one", "two", "three"],
                                   name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [4]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [5]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [11]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])

In [12]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [13]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [14]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [15]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [17]:
df = pd.DataFrame({"left": result, "right": result + 5},
              columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [18]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [19]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" Format

In [38]:
data = pd.read_csv("examples/macrodata.csv")

In [39]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

In [40]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [41]:
periods = pd.PeriodIndex(year=data.pop("year"),
                        quarter=data.pop("quarter"),
                        name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [42]:
data.index = periods.to_timestamp("D")

In [43]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [44]:
data= data.reindex(columns=["readlgdp", "infl", "unemp"])

In [45]:
data.columns.name = "item"

In [46]:
data.head()

item,readlgdp,infl,unemp
date,,,
1959-01-01,NaN,0.00,5.8
1959-04-01,NaN,2.34,5.1
1959-07-01,NaN,2.74,5.3
1959-10-01,NaN,0.27,5.6
1960-01-01,NaN,2.31,5.2


In [47]:
long_data = (data.stack()
            .reset_index()
            .rename(columns={0: "value"}))

In [48]:
long_data[:10]

,date,item,value
0,1959-01-01,infl,0.00
1,1959-01-01,unemp,5.80
2,1959-04-01,infl,2.34
3,1959-04-01,unemp,5.10
4,1959-07-01,infl,2.74
5,1959-07-01,unemp,5.30
6,1959-10-01,infl,0.27
7,1959-10-01,unemp,5.60
8,1960-01-01,infl,2.31
9,1960-01-01,unemp,5.20


In [51]:
pivoted = long_data.pivot(index="date", columns="item", values="value")

In [52]:
pivoted.head()

item,infl,unemp
date,,
1959-01-01,0.00,5.8
1959-04-01,2.34,5.1
1959-07-01,2.74,5.3
1959-10-01,0.27,5.6
1960-01-01,2.31,5.2


In [56]:
long_data["value2"] = np.random.standard_normal(len(long_data))

In [57]:
long_data[:10]

,date,item,value,value2
0,1959-01-01,infl,0.00,-0.673939
1,1959-01-01,unemp,5.80,-0.408945
2,1959-04-01,infl,2.34,1.198627
3,1959-04-01,unemp,5.10,0.799144
4,1959-07-01,infl,2.74,0.801651
5,1959-07-01,unemp,5.30,-0.616189
6,1959-10-01,infl,0.27,0.772115
7,1959-10-01,unemp,5.60,-1.063677
8,1960-01-01,infl,2.31,0.192205
9,1960-01-01,unemp,5.20,-1.218927


In [58]:
pivoted = long_data.pivot(index="date", columns="item")

In [59]:
pivoted.head()

value          value2          
item        infl unemp      infl     unemp
date                                      
1959-01-01  0.00   5.8 -0.673939 -0.408945
1959-04-01  2.34   5.1  1.198627  0.799144
1959-07-01  2.74   5.3  0.801651 -0.616189
1959-10-01  0.27   5.6  0.772115 -1.063677
1960-01-01  2.31   5.2  0.192205 -1.218927

In [60]:
pivoted["value"].head()

item,infl,unemp
date,,
1959-01-01,0.00,5.8
1959-04-01,2.34,5.1
1959-07-01,2.74,5.3
1959-10-01,0.27,5.6
1960-01-01,2.31,5.2


In [61]:
unstacked= long_data.set_index(["date", "item"]).unstack(level="item")

In [63]:
unstacked.head()

value          value2          
item        infl unemp      infl     unemp
date                                      
1959-01-01  0.00   5.8 -0.673939 -0.408945
1959-04-01  2.34   5.1  1.198627  0.799144
1959-07-01  2.74   5.3  0.801651 -0.616189
1959-10-01  0.27   5.6  0.772115 -1.063677
1960-01-01  2.31   5.2  0.192205 -1.218927

### Pivoting "Wide" to "Long" Format

In [64]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                  "A": [1, 2, 3],
                  "B": [4, 5, 6],
                  "C": [7, 8, 9]})

In [65]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [66]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [67]:
reshaped = melted.pivot(index="key", columns="variable",
                       values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [68]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [69]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [70]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [71]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [72]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [74]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
